In [1]:
import os
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
from mmseg.apis import init_segmentor, inference_segmentor
import cv2
import numpy as np
import matplotlib.pyplot as plt

/home/zhangzr/anaconda3/envs/mmseg-kaggle/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up models

In [2]:
config_file = '../cache/upernet_convnext_base_fp16_512x512_160k_FTU/upernet_convnext_base_fp16_512x512_160k_FTU.py'
checkpoint_file = '../cache/upernet_convnext_base_fp16_512x512_160k_FTU/best_mDice_iter_60800.pth'
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/zhangzr/FTU_seg/mmseg/models/losses/cross_entropy_loss.py:242: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: ../cache/upernet_convnext_base_fp16_512x512_160k_FTU/best_mDice_iter_60800.pth


In [3]:
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Make test submission csv

In [6]:
data_dir = "../data/hubmap-organ-segmentation"
test_dir = os.path.join(data_dir, "test_images")
sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
test_images = glob.glob(os.path.join(test_dir, "**", "*.tiff"), recursive = True)
sub

,id,rle
0,10078,12 34


# predict

In [15]:

for index,img_name in enumerate(test_images):
    id = img_name.split('/')[-1].split('.')[0]
    res = inference_segmentor(model, img_name)[0]
    rle = rle_encode(res.squeeze())
    sub.loc[index, "id"] = id
    sub.loc[index, "rle"] = rle



# Format submission

In [16]:
sub.to_csv("submission.csv", index = False)
sub

,id,rle
0,10078,1374938 6 1376952 18 1378973 21 1380952 1 1380...
